In [ ]:
!git clone https://github.com/sskyisthelimit/DS_Internship.git

In [3]:
cd DS_Internship/NLP/

/content/DS_Internship/NLP


In [ ]:
!pip install -r requirements.txt

In [5]:
cd src

/content/DS_Internship/NLP/src


In [ ]:
import json
# loading test dataset from repo
with open("../datasets/test_dataset.json", "r") as f:
  test_samples = json.load(f)

In [10]:
from inference import (post_process_tokens_and_labels,
                       predict_ner_labels, labels, label2id, id2label,
                       AutoTokenizer, AutoModelForTokenClassification)

In [11]:
model_name = "sskyisthelimit/mount-ner-model"
ch_model_base = "e1c89f5ecd230ded080eebefecf5c3cfbcc2adf4"
ch_tokenizer_base = "3812b840d1845edc8bc80585d53575ae2f08340f"

ch_model_ner_specific = "02f7f3f85bb8ad67056d65f329318c78a491094f"
ch_tokenizer_ner_specific = "7007bb97f0244bce71016c08fc204a3581b8cdd5"

tokenizer_base = AutoTokenizer.from_pretrained(model_name, revision=ch_tokenizer_base)
model_base = AutoModelForTokenClassification.from_pretrained(model_name, revision=ch_model_base)

tokenizer_ner_specific = AutoTokenizer.from_pretrained(model_name, revision=ch_tokenizer_ner_specific)
model_ner_specific = AutoModelForTokenClassification.from_pretrained(model_name, revision=ch_model_ner_specific)


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/919 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [13]:
import re
from utils import convert_to_bert_format

device = "cpu"
# Converting samples to same format as outputs of my model
# code at the end of src/utils.py
formatted_test_samples = []
for sample in test_samples["annotations"]:
  formatted_test_samples.append(
      convert_to_bert_format(sample[0], sample[1]["entities"], tokenizer_base))


In [16]:
import torch
from sklearn.metrics import classification_report
import spacy
from spacy.tokens import Doc, Span
from spacy import displacy

# SpaCy setup for visualization
nlp = spacy.blank("en")  # Create a blank SpaCy pipeline

def create_spacy_doc(tokens, labels, label_map):
    """
    Create a SpaCy Doc object with labeled entities for visualization.

    Args:
    tokens (list): List of tokens.
    labels (list): List of corresponding labels.
    label_map (dict): Mapping from label IDs to label names.

    Returns:
    Doc: A SpaCy Doc object with labeled entities.
    """
    doc = Doc(nlp.vocab, words=tokens)
    entities = []
    for i, label in enumerate(labels):
        if label > 0:  # Assuming 0 is 'O' or non-entity
            entities.append(Span(doc, i, i + 1, label=label_map[label]))
    doc.ents = entities
    return doc



def compute_metrics_and_visualize(formatted_test_samples, model, tokenizer, device, label_map):
    """
    Compute metrics and visualize problematic sentences.

    Args:
    formatted_test_samples (list): List of tuples containing tokens, labels, and sentences.
    model (torch.nn.Module): The fine-tuned NER model.
    tokenizer (transformers.PreTrainedTokenizer): The tokenizer used to tokenize the input.
    device (torch.device): The device (CPU or GPU) on which to run the model.
    label_map (dict): Mapping from label IDs to label names.
    """
    y_true, y_pred = [], []

    for tokens, labels, sentence in formatted_test_samples:
        # Predict labels for the sentence
        pred_tokens, pred_labels = predict_ner_labels(sentence, model, tokenizer, device, False)
        tokens, labels = post_process_tokens_and_labels(tokens, labels)

        # Adjust labels for equivalence rules
        adjusted_preds = [
            true if (label == true or (label, true) in [(3, 1), (1, 3), (2, 4), (4, 2)]) else 0
            for label, true in zip(pred_labels, labels)
        ]

        y_true.extend(labels)
        y_pred.extend(adjusted_preds)
        # Disabled to check classification report

        # Check for problematic sentences
        # if labels != adjusted_preds:
        #     print("Problematic Sentence:")
        #     print("Original:")
        #     displacy.render(create_spacy_doc(tokens, labels, label_map), style="ent", jupyter=True)
        #     print("Predicted:")
        #     displacy.render(create_spacy_doc(pred_tokens, adjusted_preds, label_map), style="ent", jupyter=True)

    # Compute metrics
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=list(label_map.values())))


compute_metrics_and_visualize(formatted_test_samples, model_base, tokenizer_base, device, id2label)
compute_metrics_and_visualize(formatted_test_samples, model_ner_specific, tokenizer_ner_specific, device, id2label)


Classification Report:
              precision    recall  f1-score   support

           O       0.98      1.00      0.99      1040
  B-MOUNTAIN       1.00      0.85      0.92        55
  I-MOUNTAIN       1.00      0.92      0.96        38
 B-ELEVATION       1.00      0.54      0.70        13
 I-ELEVATION       1.00      0.67      0.80         6

    accuracy                           0.98      1152
   macro avg       1.00      0.80      0.87      1152
weighted avg       0.98      0.98      0.98      1152

Classification Report:
              precision    recall  f1-score   support

           O       0.97      1.00      0.99      1040
  B-MOUNTAIN       1.00      0.73      0.84        55
  I-MOUNTAIN       1.00      0.84      0.91        38
 B-ELEVATION       1.00      0.54      0.70        13
 I-ELEVATION       1.00      0.67      0.80         6

    accuracy                           0.97      1152
   macro avg       0.99      0.75      0.85      1152
weighted avg       0.98      0.

Conclusion: base, not NER specific base-cased BERT provides better results in this case, classification report shows that. 